<a href="https://colab.research.google.com/github/DB-Schmudde/AI2/blob/main/AI2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install optuna

In [ ]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from optuna import trial
import numpy as np
from tqdm import tqdm
import optuna

num_epochs = 3

BATCH_SIZE = 64

train_dataset = torchvision.datasets.MNIST(root='./data',
                                           download=True,
                                           train=True,
                                           transform=transforms.ToTensor())

test_dataset = torchvision.datasets.MNIST(root='./data',
                                          download=True,
                                          transform=transforms.ToTensor())


In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=BATCH_SIZE,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=BATCH_SIZE)

In [ ]:
def objective(trial):
  n_layers = trial.suggest_int('layers',1,5)
  hidden_size = trial.suggest_int('nodes',50,200)
  num_classes = 10
  input_size = 784
  lr = trial.suggest_float('learning_rate',1e-4,1e-3)

  class Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
      super().__init__()

      self.input_size = input_size
      self.l1 = nn.Linear(self.input_size, hidden_size)
      self.l2 = nn.Linear(hidden_size, hidden_size)
      self.l3 = nn.Linear(hidden_size, num_classes)

      def forward(self,x):
        out = self.l1(x)
        for i in range(n_layers):
          out = torch.nn.ReLU()(self.l2(out))
        out = self.l3(out)
        return out

        model = Model(input_size, hidden_size, num_classes).cuda()

        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(),lr = lr)

        for epoch in range(num_epochs):
          ep_loss = 0.0
          for batch in tqdm(train_loader):
            images, labels = batch
            images = images.reshape(-1,28*28).cuda()
            labels = labels.cuda()

            out = model(images)
            loss = criterion(out, labels)
            ep_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f"epoch loss was{ep_loss}")
      return ep_loss

In [ ]:
study = optuna.create_study(direction= "minimize")
study.optimize(objective, n_trials = 10)



[I 2024-12-27 18:00:21,180] A new study created in memory with name: no-name-67ca68ad-0454-4883-8437-c7751fda671a
[W 2024-12-27 18:00:21,189] Trial 0 failed with parameters: {'layers': 4, 'nodes': 71, 'learning_rate': 0.0007085678707713514} because of the following error: The value None could not be cast to float..
[W 2024-12-27 18:00:21,192] Trial 0 failed with value None.
[W 2024-12-27 18:00:21,198] Trial 1 failed with parameters: {'layers': 1, 'nodes': 82, 'learning_rate': 0.0001636707466357294} because of the following error: The value None could not be cast to float..
[W 2024-12-27 18:00:21,203] Trial 1 failed with value None.
[W 2024-12-27 18:00:21,205] Trial 2 failed with parameters: {'layers': 3, 'nodes': 92, 'learning_rate': 0.00019703699516845067} because of the following error: The value None could not be cast to float..
[W 2024-12-27 18:00:21,211] Trial 2 failed with value None.
[W 2024-12-27 18:00:21,214] Trial 3 failed with parameters: {'layers': 1, 'nodes': 191, 'learnin